# Export Deepseg

## Export Bounding Circle Detection

In [ ]:
import sys
sys.path.append('/home/martin/Dev/homography_imitation_learning')

import os
import torch
from lightning_modules import ImageSegmentationModule

from utils import load_yaml, generate_path

devices = ['cpu', 'cuda']
output_path = '/tmp/models'
generate_path(output_path)
for device in devices:
    # load model with default values
    # prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/resnet/34/tiny'
    # checkpoint_prefix = 'version_5'
    # checkpoint = 'checkpoints/epoch=374-step=1499.ckpt'
    prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/unet/resnet/34'
    checkpoint_prefix = 'version_4'
    checkpoint = 'checkpoints/epoch=288-step=2311.ckpt'

    config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

    model = ImageSegmentationModule.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
    model = model.to(device)

    script = model.to_torchscript(method='trace', example_inputs=torch.rand([1,3,240,320]))
    script = torch.jit.optimize_for_inference(script)
    # torch.jit.save(script, f'{output_path}/seg_unet_resnet_34_{device}_tiny.pt')
    torch.jit.save(script, f'{output_path}/seg_unet_resnet_34_{device}.pt')

## Export Deep Homography Estimation

In [ ]:
import sys
sys.path.append('/home/martin/Dev/homography_imitation_learning')

import os
import torch
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

from utils import load_yaml, generate_path

devices = ['cpu', 'cuda']
output_path = '/tmp/models'
generate_path(output_path)
for device in devices:
    # load model with default values
    prefix = '/home/martin/Tresors/homography_imitation_learning_logs/ae_cai/resnet/48/25/34'
    checkpoint_prefix = 'version_0'
    checkpoint = 'checkpoints/epoch=99-step=47199.ckpt'

    config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

    model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
    model = model.to(device)

    script = model.to_torchscript(method='trace', example_inputs=[torch.rand([1,3,240,320]), torch.rand([1,3,240,320])])
    script = torch.jit.optimize_for_inference(script)
    torch.jit.save(script, f'{output_path}/h_est_48_resnet_34_{device}.pt')

## Export Homography Imitation

In [ ]:
import sys
sys.path.append('/home/martin/Dev/homography_imitation_learning')

import os
import torch
from lightning_modules import FeatureLSTMModule

from utils import load_yaml, generate_path

devices = ['cpu', 'cuda']
output_path = '/tmp/models'
generate_path(output_path)
for device in devices:
    # load model with default values
    prefix = '/home/martin/Tresors/homography_imitation_learning_logs/miccai/feature_lstm/phantom/resnet34/frame_increment_1/transforms'
    checkpoint_prefix = 'version_0'
    checkpoint = 'checkpoints/epoch=15-step=320.ckpt'

    config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

    model = FeatureLSTMModule.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config["model"])
    model = model.to(device)
    model.freeze()

    script = model.to_torchscript(method='trace', example_inputs=[torch.rand([1,1,3,240,320]), torch.rand([1,1,4,2]), torch.rand([1,1,4,2])])
    script = torch.jit.optimize_for_inference(script)
    torch.jit.save(script, f'{output_path}/h_pred_feature_lstm_{device}.pt')
